<a href="https://colab.research.google.com/github/kokchun/Databehandling-21/blob/main/Lectures/L7-high-performance.ipynb" target="_parent"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; for interacting with the code


---
# Lecture notes - High performance Pandas
---

This is a lecture note on **high performance Pandas** - but it's built upon contents from pandas and previous course:

- Python programming

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives a brief introduction to high performance. I encourage you to read further about high performance.

Read more

- [Enhancing performance](https://pandas.pydata.org/docs/user_guide/enhancingperf.html)
- [pandas eval()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.eval.html?highlight=eval#pandas.DataFrame.eval)
- [pandas query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html?highlight=query#pandas.DataFrame.query)
- [Scaling to large datasets](https://pandas.pydata.org/docs/user_guide/scale.html?highlight=efficency)

---


## Eval

We use a compound expression to motivation eval(): 复合表达式

```python
mask = (x > 0.5) & (y < 0.5)
```
will create the following steps which are explicitly allocated in memory: 
将创建以下在内存中显式分配的步骤：

```python
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2
```

Using eval() will perform elementwise directly without intermediate steps using numexpr. 
使用 eval() 将直接按元素执行，无需使用 numexpr 的中间步骤。

eval can be slower than normal pandas expressions. Rule of thumb:
if df rows > 10000 can use eval() else use normal df expressions


eval 可能比普通的熊猫表达式慢。 经验法则：
如果 df rows> 10000 可以使用 eval() else 使用普通的 df 表达式

In [2]:
import numpy as np 
import pandas as pd 

nrows, ncols = 1000000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.205457,-0.182551,-0.009061,-0.699875,0.751593,0.474389,0.043159,-1.271642,0.754777,1.132855,...,-0.293654,1.612090,-1.475015,-1.209662,-0.925129,0.460353,0.257400,-0.917430,-0.267208,0.857443
1,-0.531015,-0.946944,0.559725,-0.069262,-0.771186,-1.017211,0.059933,1.533644,0.554080,-0.964411,...,0.391085,-1.522373,0.390564,-1.012923,-1.069849,-0.561631,-0.567942,0.845234,-0.513721,-1.538440
2,0.531290,-1.141891,0.895088,-0.975716,-0.126185,1.325676,0.890158,-0.732867,-1.091721,0.515648,...,0.418763,-1.023120,-1.299288,0.907884,0.780402,-0.703254,0.192049,-0.653800,-0.702207,1.320465
3,-0.410395,0.688841,-1.785622,-0.792788,0.931509,0.007275,1.340196,1.120139,0.204782,0.264155,...,0.024022,0.048829,0.284419,-0.247120,-0.168281,-0.610747,0.105849,0.217055,-0.066580,0.615990
4,-0.745411,0.838125,-1.626893,-0.439658,0.746300,0.590884,-2.126536,0.455212,-0.153886,-1.392623,...,0.019358,-0.364833,0.419621,0.653097,2.893194,-0.042056,-1.073423,1.376996,-0.538776,-0.375889


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [5]:
%timeit df1+df2+df3+df4
%timeit sum(df1+df2+df3+df4)

The slowest run took 5.87 times longer than the fastest. This could mean that an intermediate result is being cached.
4.8 s ± 3.98 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.89 s ± 468 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit pd.eval( "df1+df2+df3+df4")

In [ ]:
plain=df1+df2+df3+df4
sum_eval=pd.eval("df1+df2+df3+df4")
sum_eval.equals(plain)

In [7]:
rolls=pd.DataFrame(np.random.randint(1,6,(6,3)),columns=["Die1","Die2","Die3"])
rolls.eval("Sum=Die1+Die2+Die3",inplace=True)
rolls

,Die1,Die2,Die3,Sum
0,3,5,2,10
1,2,2,4,8
2,4,1,3,8
3,2,4,3,9
4,4,2,2,8
5,5,1,3,9


In [8]:
# use variables 
high=10
rolls.eval("Winner = Sum > @high",inplace=True)
rolls

,Die1,Die2,Die3,Sum,Winner
0,3,5,2,10,False
1,2,2,4,8,False
2,4,1,3,8,False
3,2,4,3,9,False
4,4,2,2,8,False
5,5,1,3,9,False


In [9]:
#filter out "traditional" way
rolls[rolls["Sum"]<=high]

,Die1,Die2,Die3,Sum,Winner
0,3,5,2,10,False
1,2,2,4,8,False
2,4,1,3,8,False
3,2,4,3,9,False
4,4,2,2,8,False
5,5,1,3,9,False


# Query

In [10]:
rolls.query("Sum<=@high")

,Die1,Die2,Die3,Sum,Winner
0,3,5,2,10,False
1,2,2,4,8,False
2,4,1,3,8,False
3,2,4,3,9,False
4,4,2,2,8,False
5,5,1,3,9,False


#另外一种表达式

In [12]:
import time 
DataFrame_1=pd.DataFrame(np.random.randint(0,10000,(1000,1000)))
DataFrame_2=pd.DataFrame(np.random.randint(0,10000,(1000,1000)))
DataFrame_3=pd.DataFrame(np.random.randint(0,10000,(1000,1000)))
DataFrame_4=pd.DataFrame(np.random.randint(0,10000,(1000,1000)))

start_1=time.clock()
DataFrame_1 + DataFrame_2 + DataFrame_3 + DataFrame_4
end_1=time.clock()
print(end_1 - start_1)

start_2=time.clock()
pd.eval(" DataFrame_1 + DataFrame_2 + DataFrame_3 + DataFrame_4 ")
end_2=time.clock()
print(end_2 - start_2) #比较原始方法和EVAL 方法需要的时间

#我们可以看到,在处理1000×1000的数组时,使用eval函数就将性能提升了三倍多一点

#所以在一般情况下,在处理大型数据时,我们最好使用eval或者query这样的高性能运算来节约运算时间

AttributeError: module 'time' has no attribute 'clock'

In [1]:
# pd.eval()
%timeit sum_plain = df1+df2+df3+df4
%timeit sum_eval = pd.eval("df1 + df2 + df3 + df4")
sum_plain = df1+df2+df3+df4
sum_eval = pd.eval("df1 + df2 + df3 + df4")
sum_plain.equals(sum_eval)

NameError: name 'df1' is not defined

In [6]:
# df.eval()
rolls = pd.DataFrame(np.random.randint(1,6,(6,3)), columns = ["Die1", "Die2", "Die3"])
rolls.eval("Sum = Die1 + Die2 + Die3", inplace = True)
rolls

,Die1,Die2,Die3,Sum
0,5,1,1,7
1,3,1,1,5
2,3,5,3,11
3,5,5,1,11
4,1,1,3,5
5,4,2,5,11


In [7]:
# use variables
high = 10 
rolls.eval("High = Sum > @high", inplace = True)
rolls

,Die1,Die2,Die3,Sum,High
0,5,1,1,7,False
1,3,1,1,5,False
2,3,5,3,11,True
3,5,5,1,11,True
4,1,1,3,5,False
5,4,2,5,11,True


## Query

Cleaner syntax for selection. Faster for larger datasets and compound expressions.

用于选择的更简洁的语法。 对于更大的数据集和复合表达式，速度更快。

In [10]:
low = 10
small_plain = rolls[rolls["Sum"] < low]
small_plain

,Die1,Die2,Die3,Sum,High
0,5,1,1,7,False
1,3,1,1,5,False
4,1,1,3,5,False


In [11]:
small_query = rolls.query("Sum < @low")
small_query

,Die1,Die2,Die3,Sum,High
0,5,1,1,7,False
1,3,1,1,5,False
4,1,1,3,5,False


In [5]:
import pandas as pd 
os=pd.read_csv("../Labs/Projekt/athlete_events.csv")
os.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [6]:
%timeit os[os["Season"] == "Winter"]
%timeit os.query("Season == 'Winter'")

plain = os[os["Season"] == "Winter"]
query = os.query("Season == 'Winter'")

plain.equals(query)

43.7 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
35.1 ms ± 5.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


True

In [16]:
%timeit os[os["NOC"]=="SWE"]
%timeit os.query("NOC=='SWE'")
#比较两种方法使用时间，query更快

39.6 ms ± 8.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
26.6 ms ± 14.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%timeit os[os["Height"] > 180]
%timeit os.query("Height > 180") # note that query is slower here
#这里query 反而慢一些

22.1 ms ± 927 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
27.3 ms ± 796 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
# query faster on compound expressions as it doesn't have to save intermediate results into memory
# 在复合表达式上query更快，因为它不必将中间结果保存到内存中

%timeit os[(os["Sex"] == "F") & (os["Height"] > 180) & (os["NOC"] == "SWE")]
%timeit os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'") 

52.3 ms ± 4.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
28.5 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
plain = os[os["Height"] > 180]
query = os.query("Height > 180") 

plain.equals(query)

True

In [21]:
plain = os[(os["Sex"] == "F") & (os["Height"] > 180) & (os["NOC"] == "SWE")]
query = os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'")

plain.equals(query)

True

# pandas.DataFrame.eval函数详解
语法格式:
DataFrame.eval(expr,inplace=False,**kwargs)[source]

eval函数的常见用法
将字符串对象转换为具体对象

       1 将字符串列表转换为列表

       2 将字符串元组转换为元组
       
       3 将字符串字典转换为字典


# 1 将字符串列表转换为列表

In [34]:
a="[[1,2],[3,4],[5,6]]" #必须有双引号，#TypeError: eval() arg 1 must be a string, bytes or code object
a

'[[1,2],[3,4],[5,6]]'

In [35]:
type(a)

str

In [36]:
b=eval(a)
b

[[1, 2], [3, 4], [5, 6]]

In [37]:
type(b)

list

# 2 将字符串元组转换为元组

In [39]:

c="([1,2],[3,4],[5,6])" 
d=eval(c)
d 


([1, 2], [3, 4], [5, 6])

In [40]:
type(c),type(d)

(str, tuple)

# 3 将字符串字典转换为字典

In [48]:
s="{1:'amber',2:'andy'}"
s

"{1:'amber',2:'andy'}"

In [49]:
type(s)

str

In [50]:
t=eval(s)
t

{1: 'amber', 2: 'andy'}

In [51]:
type(t)

dict

In [60]:
rmb=input("请输入人民币金额:") #返回字符型
print(type(rmb))
rmb=eval(rmb) #转换为数字
print(type(rmb))

<class 'str'>
<class 'int'>


In [64]:
rmb=input("请输入人民币金额:") #返回字符型
rmb=eval(rmb) #将字符串转换为数字
usd_VS_rmb=6.8
usd=rmb/usd_VS_rmb
print(f"美元金额是{usd} usd")


美元金额是100.0 usd


In [1]:
import numpy as np 
rng=np.random.RandomState(42)
x=rng.rand(1000000) #1E6=1后面有6个0
x

array([0.37454012, 0.95071431, 0.73199394, ..., 0.41807198, 0.42867126,
       0.92944855])

In [2]:
y=rng.rand(1000000)
y

array([0.59515562, 0.36471714, 0.00537562, ..., 0.68311082, 0.33865907,
       0.69161641])

In [3]:
%timeit x+y

4.98 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
#这样做比普通的Python 循环或列表综合要快很多：
%timeit np.fromiter((xi+yi for xi, yi in zip(x,y)),dtype=x.dtype)

287 ms ± 39.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
#但这种运算在处理复合表达式的问题时，效率会变得不那么高效。例如，考虑以下表达式：
mask=(x>0.5) &(y<0.5)
mask

array([False,  True,  True, ..., False, False, False])

In [6]:
#因为 NumPy 会对每个代数子表达式进行计算，所以大致等价于以下内容：
tmp1=(x>0.5)
tmp2=(y<0.5)
mask=tmp1 & tmp2
mask

array([False,  True,  True, ..., False, False, False])

In [8]:
import numexpr
mask_numexpr=numexpr.evaluate("( x>0.5 ) & ( y<0.5 )")
np.allclose(mask,mask_numexpr)

ModuleNotFoundError: No module named 'numexpr'

# https://zhuanlan.zhihu.com/p/74511940

In [54]:
# import numpy as np
x=rng.rand(1E6)
y=rng.rand(1E6)
​
#mask = (x > 0.5) & (y < 0.5)

SyntaxError: invalid non-printable character U+200B (Temp/ipykernel_5588/1346758147.py, line 4)